In [6]:
#import library yang digunakan
import csv
import pandas as pd
from itertools import combinations
from time import time

#Membaca data yang akan digunakan
df = pd.read_csv('dataset.csv', sep=";")
df.head()

#Preprocessing data membuang kolom yang tidak diperlukan
grouped_df = df.groupby(by = ['TID'])
 
menu = []
for group, pdf in grouped_df:
    menu.append(pdf['Menu'].values.tolist())

#Mengubah list menjadi dataframe
data = pd.DataFrame(menu)

#Input nilai minimum support dan confidence
minsup=float(input("Masukkan nilai minimum support: "))
minsup=minsup*len(data)
minconf=float(input("Masukkan nilai minimum confidence: "))

#Mulai menghitung execution time
t1 = time()

#Data preprocessing merubah data menjadi list
items = []  
for i in range(0, len(data)):  
    items.append([str(data.values[i,j]) for j in range(0, len(data.values[0]))])

#Membuat daftar kosakata yang terdapat pada list
count = [dict() for x in range(len(data.values[0])+1)]

#Menghitung nilai support item secara satuan
s=[]
for i in items:
    for j in i:
        s.append(j)
for i in s:
    #Jika item terdapat dalam daftar kosakata, tambahkan 1 hitungan
    if i in count[1]:
        count[1][i] = count[1][i] + 1
    #Jika item tidak terdapat dalam daftar kosakata, hitung menjadi 1
    else:
        count[1][i] = 1
        
#Menghapus data yang kosong(nan values) dan infrequent items
for i in count[1].copy():
    if(count[1][i]<minsup):
        count[1].pop(i)
count[1].pop("None")

#Menyimpan transaksi yang baru tanpa infrequent items
a=list(count[1])
item=[list() for i in range(len(data))]
c=0
for i in range(0,len(items)):
    for j in range(len(items[i])):
        if(a.__contains__(items[i][j])!=0):
            item[i].append(items[i][j])

#Fungsi untuk mengurutkan list berdasarkan nilai support
def sort(a):
    for i in range(len(a)-1):
        for j in range(len(a)-i-1):
            if(count[1][a[j]]<count[1][a[j+1]]):
                a[j],a[j+1]=a[j+1],a[j]

#Memanggil fungsi untuk mengurutkan list secara descending berdasarkan nilai support
for i in range(0,len(data)):
    if(len(item[i])>1):
        sort(item[i])

#Generate frequent two itemsets
slist=[list() for x in range(33)]
a=[]
a=combinations(count[1],2)
for j in a:
    #slist[2]=list(combinations(count[1],2))
    slist[2].append(tuple(sorted(j)))    
candidates=[]
for i in slist[2]:
    candidates.append(i)
for i in candidates:
    for k in items:
        f=0
        for l in i:
            if(k.__contains__(l)==0):
                f=1
                break
        if(f==0):
            if i in count[2]:
                count[2][i]=count[2][i]+1
            else:
                count[2][i]=1
for i in count[2].copy():
    if(count[2][i]<minsup):
        count[2].pop(i)

#Generate frequent itemsets dengan length z dari z-1
def freq(z):
    for i in count[z-1]:
        for j in count[z-1]:
            a=set(i)
            b=set(j)
            #Generate z length itemsets dari z-1 length, dimama frequent itemsets memiliki z-2 elemen yang bersimpangan
            if(len(a.intersection(b))==z-2):
                #Cek jika semua subsets berada dalam frequent itemsets z-1, jika tidak maka tidak frequent
                t=list(combinations(sorted(a.union(b)), z-1))
                c=0
                for n in t:
                    for m in count[z-1]:
                        if((n)==m):
                            c=c+1
                if(c==z):
                    flag=0
                    for h in slist[z]:
                        if(sorted(list(a.union(b)))==sorted(h)):
                            flag=1
                    if(flag==0):
                        slist[z].append(tuple(sorted(list(a.union(b)))))
    #Menghitung nilai support
    candidates=[]
    for i in slist[z]:
        candidates.append(tuple(i))
    for i in candidates:
        for k in items:
            f=0
            for l in i:
                if(k.__contains__(l)==0):
                    f=1
                    break
            #Jika sebuah item lengkap ada pada transaksi, maka nilai supportnya ditambahkan
            if(f==0):
                #Jika sudah terdapat pada daftar kosakata maka tambahkan nilainya 1
                if i in count[z]:
                    count[z][i]=count[z][i]+1
                #Jika belum ada tambahkan pada daftar kosakata
                else:
                    count[z][i]=1
    for i in count[z].copy():
        if(count[z][i]<minsup):
            count[z].pop(i)

#Memanggil fungsi untuk generate frequent itemssets
i=3
while(len(count[i-1])!=0):
    freq(i)
    i=i+1
q=i-2

#Fungsi untuk ekstraksi sebuah itemset dari tuple
def value(a):
    a=str(a)
    a=a[:-2]
    a=a[2:]
    n=a[:-1]
    return n

#Mencari itemset yang maksimal dan dekat
maximal=[]
closed=[]
for i in range(1,q):
    for j in count[i]:
        fm=0
        fc=0
        for k in count[i+1]:
            a=set(list([j]))
            b=set(list(k))
            #Set menjadi maksimal jika tidak ada superset yang frequent
            if(a.intersection(b)==a):
                fm=1
                #Set dekat jika superset tidak memiliki nilai support yang sama
                if(count[i][j]==count[i+1][k]):
                    fc=1
        if(fm==0):
            maximal.append(j)
        if(fc==0):
            closed.append(j)
#Semua set yang terdapat pada pohon bagian atas, otomatis menjadi itemset yang maksimal dan dekat
for i in count[q]:
    maximal.append(i)
    closed.append(i)

#Menemukan Association Rules 
c=0
a=[]
item_a=[]
support_a=[]
b=[]
item_b=[]
support_b=[]
support=[]
confidence=[]
lift_ratio=[]
ant=count.copy()
for i in range(q,0,-1):
    for j in ant[i]:
        for k in range(i-1,0,-1):
            s=list(combinations(list(j),k))      
            #Membaca semua daftar kombinasi sebelumnya
            for n in s:
                #Sorting untuk mencegah duplikasi 
                r=tuple(sorted(set(j).difference(set(n))))
                l=len(n)
                #Check if len(n)==1 agar dapat mengekstrak "key" untuk mencari di support dictionaries. 
                if(l==1):
                    n=value(n)
                    l=1
                if(len(r)==1):
                    r2=value(r)
                if(n!=None):
                    #Jika rules confidencenya lebih besar dari minconf, print the rule
                    if((ant[len(j)][j]/ant[l][n])>=minconf):
                    	#Rules akan terlihat signifikan jika terdapat pada node tertutup,jika tidak akan berlebihan
                        if(closed.__contains__((n))):
                            c=c+1
                            if(len(r)==1):
                                a.append(n)
                                item_a.append(ant[l][n])
                                support_a.append((ant[l][n]/len(data)))
                                b.append(r2)
                                item_b.append(ant[len(r)][r2])
                                support_b.append((ant[len(r)][r2]/len(data)))
                                support.append((ant[len(j)][j]/len(data)))
                                confidence.append((ant[len(j)][j]/ant[l][n]))
                                lift_ratio.append(((ant[len(j)][j])/ant[l][n])/(ant[len(r)][r2])) 
                            else:
                                a.append(n)
                                item_a.append(ant[l][n])
                                support_a.append((ant[l][n]/len(data)))
                                b.append(r)
                                item_b.append(ant[len(r)][r])
                                support_b.append((ant[len(r)][r]/len(data)))
                                support.append((ant[len(j)][j]/len(data)))
                                confidence.append((ant[len(j)][j]/ant[l][n]))
                                lift_ratio.append(((ant[len(j)][j])/ant[l][n])/(ant[len(r)][r]))

table_rules = pd.DataFrame({'Antecendent':a,
                            'Frequent_item_a':item_a,
                            'Support_item_a':support_a,
                            'Consequent':b,
                            'Frequent_item_b':item_b,
                            'Support_item_b':support_b,
                            'Support':support,
                            'Confidence':confidence,
                            'Lift_Ratio':lift_ratio
                           })

#Fungsi mengkali hasil tabel support dengan table confidence untuk keperluan menghitung kekuatan association rules
table_rules['SxC'] = table_rules['Support'] * table_rules['Confidence']

#Memanggil akhir hitungan execution time
t2 = time()
elapsed = t2 - t1

#Untuk memanggil hasil
table_rules

Masukkan nilai minimum support: 0.006
Masukkan nilai minimum confidence: 0.2


,Antecendent,Frequent_item_a,Support_item_a,Consequent,Frequent_item_b,Support_item_b,Support,Confidence,Lift_Ratio,SxC
0,"(Cookies Hias, Dimsum Siomay Udang Keju)",47,0.028589,Dimsum Siomay Ayam,308,0.187348,0.006083,0.212766,0.000691,0.001294
1,"(Cookies Hias, Dimsum Siomay Ayam)",58,0.035280,Dimsum Siomay Keju,328,0.199513,0.009124,0.258621,0.000788,0.002360
2,"(Cookies Hias, Dimsum Siomay Keju)",51,0.031022,Dimsum Siomay Ayam,308,0.187348,0.009124,0.294118,0.000955,0.002684
3,"(Dimsum Siomay Ayam, Dimsum Siomay Keju)",62,0.037713,Cookies Hias,286,0.173966,0.009124,0.241935,0.000846,0.002207
4,"(Brownies Cookies, Dimsum Siomay Udang Keju)",63,0.038321,Lavarisol Campur,276,0.167883,0.007908,0.206349,0.000748,0.001632
5,"(Brownies Cookies, Lavarisol Campur)",49,0.029805,Dimsum Siomay Udang Keju,326,0.198297,0.007908,0.265306,0.000814,0.002098
6,"(Dimsum Siomay Udang Keju, Lavarisol Campur)",53,0.032238,Brownies Cookies,303,0.184307,0.007908,0.245283,0.000810,0.001940
7,"(Blueberry Cheese, Brownies Cookies)",50,0.030414,Dimsum Siomay Udang Keju,326,0.198297,0.007299,0.240000,0.000736,0.001752
8,"(Blueberry Cheese, Dimsum Siomay Udang Keju)",48,0.029197,Brownies Cookies,303,0.184307,0.007299,0.250000,0.000825,0.001825
9,"(Brownies Cookies, Sagu Keju)",48,0.029197,Dimsum Siomay Udang Keju,326,0.198297,0.006083,0.208333,0.000639,0.001267


In [ ]:
print('Execution time : %f detik.' % elapsed)

In [ ]:
#Fungsi untuk menghitung kekuatan association rules
kekuatan = sum(table_rules['SxC']) / len(table_rules)

print ("Kekuatan Association Rules Algoritma Apriori : ", kekuatan )


In [ ]:
#Fungsi untuk menghitung jumlah nilai support dari hasil guna perhitungan akurasi algoritma
jumlah_support = sum(table_rules['Support'])

print ("Jumlah Nilai Support Algoritma Apriori : ", jumlah_support )